<a href="https://colab.research.google.com/github/JoshuaStorm1017/FreeTranscriber/blob/main/FT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 🎙️ Easy Audio/Video Transcriber (with URL & YouTube support) 📝
#@markdown 1. Run this cell (click the "play" button or press Shift+Enter).
#@markdown 2. It will install necessary tools (may take a minute or two the first time).
#@markdown 3. Choose your **Input Method**: "Upload File" or "Enter URL".
#@markdown 4.  - If "Upload File": Click "Choose Files" to upload your file.
#@markdown     - If "Enter URL": Paste the direct link to your audio/video file (or a YouTube video URL) in the "File URL" box.
#@markdown 5. Select the **Media Type** (Audio or Video). For YouTube URLs, this is less critical as we'll try to get audio directly.
#@markdown 6. Click the "Transcribe" button.
#@markdown 7. Wait for the magic! The transcript will appear below, and a download link for a .txt file will be provided.

# --- 1. Install necessary libraries ---
# -q for "quiet" installation
print("Installing necessary libraries... This might take a minute or two the first time.")
!pip install -q openai-whisper moviepy requests yt-dlp
!apt-get -qq install ffmpeg # Ensure ffmpeg is available for moviepy & yt-dlp

import whisper
import moviepy.editor as mp
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from google.colab import files
import os
import datetime
import requests # For downloading from general URLs
import urllib.parse # For parsing URLs
import torch # To check for GPU availability for fp16
import subprocess # To run yt-dlp
import re # For simple YouTube URL check

# --- 2. Global variables and helper functions ---
MODEL_NAME = "small" # "tiny", "base", "small", "medium", "large". "small" is a good balance.
UPLOADED_FILE_INFO = {}
DOWNLOADED_FILE_PATH_FROM_URL = None
YT_DOWNLOADED_AUDIO_PATH = None # Specifically for audio downloaded from YouTube
TRANSCRIPT_TEXT = ""
model = None

def load_whisper_model():
    global model
    if model is None:
        print(f"Loading Whisper model ('{MODEL_NAME}')... This can take some time, especially for larger models.")
        try:
            model = whisper.load_model(MODEL_NAME)
            print(f"Whisper model '{MODEL_NAME}' loaded successfully!")
        except Exception as e:
            print(f"Error loading Whisper model: {e}")
            print("This might be due to insufficient RAM/GPU memory. Try a smaller model (e.g., 'tiny' or 'base')")
            print("Or, go to Runtime > Change runtime type and ensure a GPU is selected (if available).")
            model = None
    return model

def get_formatted_timestamp():
    return datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

def get_filename_from_url(url):
    try:
        parsed_url = urllib.parse.urlparse(url)
        filename = os.path.basename(parsed_url.path)
        if not filename:
            content_type = requests.head(url, timeout=10, allow_redirects=True).headers.get('content-type', '').split('/')[0]
            extension_map = {'audio': 'mp3', 'video': 'mp4', 'application': 'bin'}
            ext = extension_map.get(content_type, 'dat')
            filename = f"downloaded_media_{get_formatted_timestamp()}.{ext}"
        # Ensure filename doesn't have query parameters if path was short
        filename = filename.split('?')[0]
        return filename
    except Exception:
        return f"downloaded_media_{get_formatted_timestamp()}"

def is_youtube_url(url):
    # Simple regex to check for common YouTube URL patterns
    youtube_regex = (
        r'(https?://)?(www\.)?'
        '(youtube|youtu|youtube-nocookie)\.(com|be)/'
        '(watch\?v=|embed/|v/|.+\?v=)?([^&=%\?]{11})')
    return re.match(youtube_regex, url)

# --- 3. Widget Setup ---
# (Widget setup remains largely the same as your previous version)
input_method_radio = widgets.RadioButtons(
    options=['Upload File', 'Enter URL'],
    description='Input Method:',
    value='Upload File',
    disabled=False,
)

file_type_dropdown = widgets.Dropdown(
    options=['Audio', 'Video'],
    value='Video', # Default to video as YT links are usually video
    description='Media Type:',
    disabled=False,
)

file_uploader = widgets.FileUpload(
    accept='.mp3,.wav,.m4a,.ogg,.flac,.mp4,.mov,.avi,.mkv',
    multiple=False,
    description='Choose File',
    layout={'display': 'flex'}
)

url_input_text = widgets.Text(
    value='',
    placeholder='e.g., https://example.com/audio.mp3 or YouTube URL',
    description='File URL:',
    layout={'width': '95%', 'display': 'none'},
    disabled=False
)

transcribe_button = widgets.Button(
    description='Transcribe',
    disabled=True,
    button_style='info',
    tooltip='Upload a file or enter a URL, then click to transcribe',
    icon='microphone'
)

download_button = widgets.Button(
    description='Download Transcript (.txt)',
    disabled=True,
    button_style='success',
    tooltip='Download the transcript as a text file',
    icon='download'
)

output_area = widgets.Output()
transcription_display = widgets.Textarea(
    value='',
    placeholder='Transcription will appear here...',
    description='Transcript:',
    layout={'height': '200px', 'width': '95%'},
    disabled=True
)


# --- 4. Event Handlers ---
def on_input_method_change(change):
    global UPLOADED_FILE_INFO, DOWNLOADED_FILE_PATH_FROM_URL, YT_DOWNLOADED_AUDIO_PATH
    with output_area:
        # Don't clear output_area fully here, as model loading message might be present
        pass # Message handling will be in specific button clicks or a dedicated status area

    if change.new == 'Upload File':
        file_uploader.layout.display = 'flex'
        url_input_text.layout.display = 'none'
        DOWNLOADED_FILE_PATH_FROM_URL = None
        YT_DOWNLOADED_AUDIO_PATH = None
        if url_input_text.value: url_input_text.value = ""
        transcribe_button.disabled = not bool(UPLOADED_FILE_INFO)
        if not UPLOADED_FILE_INFO and file_uploader.value:
             with output_area: # Use with output_area to append messages
                print("Switched to 'Upload File' mode. If a file name is shown, please re-select it or choose a new file.")
    elif change.new == 'Enter URL':
        file_uploader.layout.display = 'none'
        url_input_text.layout.display = 'flex'
        UPLOADED_FILE_INFO = {}
        transcribe_button.disabled = not url_input_text.value.strip()

    download_button.disabled = True
    transcription_display.value = ""

def on_file_upload_change(change):
    global UPLOADED_FILE_INFO
    # Clearing output only if there's a new meaningful message to display
    if file_uploader.value or not UPLOADED_FILE_INFO: # if value changes or was empty
        with output_area:
            clear_output(wait=True)

    if file_uploader.value:
        uploaded_file_data_list = list(file_uploader.value.values())
        if uploaded_file_data_list:
            UPLOADED_FILE_INFO = uploaded_file_data_list[0]
            filename = UPLOADED_FILE_INFO['metadata']['name']
            with output_area: print(f"File '{filename}' selected.")
            if input_method_radio.value == 'Upload File':
                transcribe_button.disabled = False
        else:
            UPLOADED_FILE_INFO = {}
            if input_method_radio.value == 'Upload File': transcribe_button.disabled = True
            with output_area: print("File selection issue. Please try again.")
    else:
        UPLOADED_FILE_INFO = {}
        if input_method_radio.value == 'Upload File': transcribe_button.disabled = True
        with output_area: print("No file selected or selection cleared.")
    download_button.disabled = True
    transcription_display.value = ""

def on_url_input_change(change):
    if input_method_radio.value == 'Enter URL':
        transcribe_button.disabled = not change.new.strip()
    download_button.disabled = True
    transcription_display.value = ""


def on_transcribe_button_clicked(b):
    global TRANSCRIPT_TEXT, UPLOADED_FILE_INFO, DOWNLOADED_FILE_PATH_FROM_URL, YT_DOWNLOADED_AUDIO_PATH

    current_file_to_process = None
    original_filename_for_cleanup = None
    is_direct_audio_download = False # Flag to skip moviepy extraction if yt-dlp got audio

    # Disable UI
    transcribe_button.disabled = True
    download_button.disabled = True
    file_uploader.disabled = True
    url_input_text.disabled = True
    file_type_dropdown.disabled = True
    input_method_radio.disabled = True

    with output_area:
        clear_output(wait=True)

        if input_method_radio.value == 'Upload File':
            if not UPLOADED_FILE_INFO:
                print("Error: No file selected. Please select a file."); _reenable_inputs(); return
            filename = UPLOADED_FILE_INFO['metadata']['name']
            content = UPLOADED_FILE_INFO['content']
            current_file_to_process = f"/content/{filename}"
            original_filename_for_cleanup = current_file_to_process
            with open(current_file_to_process, 'wb') as f: f.write(content)
            print(f"Processing uploaded file: {filename}")

        elif input_method_radio.value == 'Enter URL':
            url = url_input_text.value.strip()
            if not url:
                print("Error: No URL provided!"); _reenable_inputs(); return

            if is_youtube_url(url):
                print(f"YouTube URL detected. Attempting to download audio with yt-dlp: {url}")
                # Define a predictable output filename for yt-dlp
                # Using video ID or title from yt-dlp could make it more unique
                # For simplicity, using a timestamped generic name for now.
                # yt-dlp can also output JSON with metadata including title, which could be used.
                yt_dlp_output_path_template = f"/content/youtube_audio_{get_formatted_timestamp()}.%(ext)s"
                # We need to find out the actual extension yt-dlp uses.
                # A common approach is to let yt-dlp choose the best audio and name it.
                # Forcing WAV for consistency with Whisper.
                YT_DOWNLOADED_AUDIO_PATH = f"/content/youtube_audio_{get_formatted_timestamp()}.wav"

                command = [
                    'yt-dlp',
                    '-x', '--audio-format', 'wav', # Extract audio, convert to WAV
                    '-o', YT_DOWNLOADED_AUDIO_PATH, # Output filename
                    '--no-playlist', # Download only the single video if it's part of a playlist
                    '--quiet', '--no-warnings', # Suppress verbose output
                    '--progress', # Show progress
                    url
                ]
                try:
                    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
                    # Print stdout/stderr line by line for progress
                    while True:
                        output_line = process.stdout.readline()
                        if output_line == '' and process.poll() is not None:
                            break
                        if output_line:
                            print(output_line.strip(), end='\r') # Use \r for progress overwrite
                    print() # Newline after progress

                    stderr_output = process.stderr.read()
                    if process.returncode != 0:
                        print(f"\nError using yt-dlp: Exit code {process.returncode}")
                        print(f"yt-dlp output:\n{stderr_output}")
                        _reenable_inputs(); return
                    if not os.path.exists(YT_DOWNLOADED_AUDIO_PATH):
                        print(f"\nError: yt-dlp ran, but output file '{YT_DOWNLOADED_AUDIO_PATH}' not found. Output:\n{stderr_output}")
                        _reenable_inputs(); return

                    print(f"Audio downloaded from YouTube successfully: {YT_DOWNLOADED_AUDIO_PATH}")
                    current_file_to_process = YT_DOWNLOADED_AUDIO_PATH
                    original_filename_for_cleanup = YT_DOWNLOADED_AUDIO_PATH
                    is_direct_audio_download = True # Mark that this is already audio
                except FileNotFoundError:
                    print("Error: yt-dlp command not found. Make sure it's installed correctly.")
                    _reenable_inputs(); return
                except Exception as e:
                    print(f"\nAn error occurred while running yt-dlp: {e}")
                    _reenable_inputs(); return
            else: # Standard URL download
                print(f"Attempting to download from general URL: {url}")
                try:
                    filename_from_url = get_filename_from_url(url)
                    DOWNLOADED_FILE_PATH_FROM_URL = f"/content/{filename_from_url}"
                    current_file_to_process = DOWNLOADED_FILE_PATH_FROM_URL
                    original_filename_for_cleanup = DOWNLOADED_FILE_PATH_FROM_URL

                    response = requests.get(url, stream=True, timeout=180, allow_redirects=True)
                    response.raise_for_status()
                    total_size = int(response.headers.get('content-length', 0))
                    dl_size = 0
                    with open(DOWNLOADED_FILE_PATH_FROM_URL, 'wb') as f:
                        for chunk in response.iter_content(chunk_size=81920):
                            f.write(chunk); dl_size += len(chunk)
                            if total_size > 0: print(f"Downloading... {dl_size/(1024*1024):.2f}MB / {total_size/(1024*1024):.2f}MB ({(dl_size/total_size)*100:.1f}%)", end='\r')
                            else: print(f"Downloading... {dl_size/(1024*1024):.2f}MB (total size unknown)", end='\r')
                    print(f"\nDownloaded successfully to: {DOWNLOADED_FILE_PATH_FROM_URL}   ")
                except requests.exceptions.RequestException as e:
                    print(f"\nError downloading from URL {url}: {e}"); _reenable_inputs(); return
                except Exception as e:
                    print(f"\nAn unexpected error occurred during download: {e}"); _reenable_inputs(); return
        else:
            print("Error: Invalid input method."); _reenable_inputs(); return

        if not current_file_to_process or not os.path.exists(current_file_to_process):
            print(f"Error: File ('{current_file_to_process}') not available for processing."); _reenable_inputs(); return

        print(f"\nStarting transcription for: {os.path.basename(current_file_to_process)}")
        print("Please be patient...")

        audio_path_to_transcribe = current_file_to_process
        selected_media_type = file_type_dropdown.value

        current_whisper_model = load_whisper_model()
        if not current_whisper_model:
            print("Transcription cannot proceed: Whisper model failed."); _reenable_inputs(True); return

        try:
            temp_extracted_audio_path = None
            # Only extract audio if it wasn't a direct audio download (like from yt-dlp)
            # AND if the user selected "Video" (or if it's a video file from generic URL)
            if not is_direct_audio_download and selected_media_type == 'Video':
                # Basic check if it might be a video file based on common extensions
                # This is imperfect but better than nothing for generic URLs
                video_extensions = ['.mp4', '.mov', '.avi', '.mkv', '.webm', '.flv']
                is_likely_video = any(current_file_to_process.lower().endswith(ext) for ext in video_extensions)

                if is_likely_video:
                    print("Extracting audio from video file...")
                    try:
                        video_clip = mp.VideoFileClip(current_file_to_process)
                        if not video_clip.audio:
                            print("Error: Video file has no audio track."); _reenable_inputs(True); return
                        base_vid_name = os.path.splitext(os.path.basename(current_file_to_process))[0]
                        temp_extracted_audio_path = f"/content/extracted_audio_{base_vid_name}_{get_formatted_timestamp()}.wav"
                        video_clip.audio.write_audiofile(temp_extracted_audio_path, codec='pcm_s16le')
                        video_clip.close(); video_clip.audio.close()
                        audio_path_to_transcribe = temp_extracted_audio_path
                        print(f"Audio extracted to: {audio_path_to_transcribe}")
                    except Exception as e_moviepy:
                        print(f"Error during MoviePy audio extraction: {e_moviepy}")
                        print("Please ensure the file is a valid video format supported by ffmpeg.")
                        _reenable_inputs(True); return
                else:
                    print(f"Media type set to 'Video' but file '{os.path.basename(current_file_to_process)}' doesn't look like a typical video. Attempting to transcribe as is.")
            elif is_direct_audio_download:
                print("Using directly downloaded audio from YouTube.")


            print(f"Transcribing '{os.path.basename(audio_path_to_transcribe)}' with Whisper model '{MODEL_NAME}'...")
            use_fp16 = torch.cuda.is_available()
            if use_fp16: print("GPU detected. Using FP16 for faster transcription.")
            else: print("No GPU detected or not using CUDA. Using FP32. This might be slower.")

            result = current_whisper_model.transcribe(audio_path_to_transcribe, fp16=use_fp16)
            TRANSCRIPT_TEXT = result["text"]

            print("\n--- Transcription Complete! ---")
            transcription_display.value = TRANSCRIPT_TEXT
            download_button.disabled = False

            if temp_extracted_audio_path and os.path.exists(temp_extracted_audio_path):
                os.remove(temp_extracted_audio_path)
                print(f"Cleaned up temporary audio file: {temp_extracted_audio_path}")

            if original_filename_for_cleanup and os.path.exists(original_filename_for_cleanup):
                if original_filename_for_cleanup != temp_extracted_audio_path : # Avoid double delete if same
                    os.remove(original_filename_for_cleanup)
                    print(f"Cleaned up processed file: {original_filename_for_cleanup}")
                    if original_filename_for_cleanup == DOWNLOADED_FILE_PATH_FROM_URL: DOWNLOADED_FILE_PATH_FROM_URL = None
                    if original_filename_for_cleanup == YT_DOWNLOADED_AUDIO_PATH: YT_DOWNLOADED_AUDIO_PATH = None

        except Exception as e:
            print(f"\n--- An error occurred during transcription: ---"); print(e)
            import traceback; traceback.print_exc()
        finally:
            _reenable_inputs(True)
            download_button.disabled = not bool(TRANSCRIPT_TEXT)

def _reenable_inputs(keep_transcribe_disabled_if_no_input=False):
    file_uploader.disabled = False
    url_input_text.disabled = False
    file_type_dropdown.disabled = False
    input_method_radio.disabled = False
    if keep_transcribe_disabled_if_no_input:
        if input_method_radio.value == 'Upload File':
            transcribe_button.disabled = not bool(UPLOADED_FILE_INFO)
        elif input_method_radio.value == 'Enter URL':
            transcribe_button.disabled = not url_input_text.value.strip()
        else: transcribe_button.disabled = True
    else: transcribe_button.disabled = False

def on_download_button_clicked(b):
    global TRANSCRIPT_TEXT
    if not TRANSCRIPT_TEXT:
        with output_area: print("No transcript available to download."); return

    base_fn = "transcription"
    if input_method_radio.value == 'Upload File' and UPLOADED_FILE_INFO and 'metadata' in UPLOADED_FILE_INFO:
        base_fn = os.path.splitext(UPLOADED_FILE_INFO['metadata']['name'])[0]
    elif input_method_radio.value == 'Enter URL' and url_input_text.value:
         # For YouTube, try to get a title if yt-dlp provided it, else use URL part
         # This part can be enhanced if yt-dlp JSON output is parsed for title
         url_fn_part = get_filename_from_url(url_input_text.value)
         # if YT_DOWNLOADED_AUDIO_PATH: # Try to get a more descriptive name
         #    url_fn_part = os.path.splitext(os.path.basename(YT_DOWNLOADED_AUDIO_PATH))[0].replace("youtube_audio_","")

         base_fn = os.path.splitext(url_fn_part)[0]


    transcript_filename = f"{base_fn}_transcript_{get_formatted_timestamp()}.txt"
    try:
        with open(transcript_filename, "w", encoding="utf-8") as f: f.write(TRANSCRIPT_TEXT)
        print(f"Transcript prepared as '{transcript_filename}'. Offering for download...")
        files.download(transcript_filename)
    except Exception as e:
        with output_area: print(f"Error preparing transcript for download: {e}")

# --- 5. Link event handlers ---
input_method_radio.observe(on_input_method_change, names='value')
file_uploader.observe(on_file_upload_change, names='value')
url_input_text.observe(on_url_input_change, names='value')
transcribe_button.on_click(on_transcribe_button_clicked)
download_button.on_click(on_download_button_clicked)

# --- 6. Display UI ---
clear_output(wait=True)
display(HTML("<h2>🎙️ Easy Audio/Video Transcriber (with URL & YouTube support) 📝</h2>"))
# (Instructions HTML as before, just ensure it mentions YouTube URLs)
display(HTML("""
<p><b>Instructions:</b></p>
<ol>
    <li>Select your <b>Input Method</b>: "Upload File" or "Enter URL".</li>
    <li>If "Upload File": Click <b>'Choose File'</b> to upload your media.</li>
    <li>If "Enter URL": Paste the direct link to your audio/video file (e.g., <code>https://example.com/my_podcast.mp3</code>) OR a YouTube video URL (e.g., <code>https://www.youtube.com/watch?v=your_video_id</code>) in the <b>'File URL'</b> box.</li>
    <li>Select the <b>Media Type</b> (Audio or Video). For YouTube URLs, we will attempt to download audio directly, so this choice is less critical for YouTube links. For other URLs, it helps determine if audio extraction is needed.</li>
    <li>Click the <b>'Transcribe'</b> button.</li>
    <li>The transcription will appear below. You can then click <b>'Download Transcript (.txt)'</b>.</li>
</ol>
<p><b>Note:</b> The first time you transcribe, the AI model needs to be downloaded. Using a GPU (Runtime > Change runtime type) is highly recommended for speed. YouTube downloads depend on video availability and network speed.</p>
"""))


ui_layout = widgets.VBox([
    input_method_radio,
    file_type_dropdown,
    file_uploader,
    url_input_text,
    widgets.HBox([transcribe_button, download_button]),
    output_area,
    transcription_display
])
display(ui_layout)

if not model:
    with output_area: load_whisper_model()

Installing necessary libraries... This might take a minute or two the first time.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 12.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.3/173.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/